In [11]:
import pandas as pd
from sklearn import datasets
import tensorflow as tf
import itertools

In [2]:
training_set = pd.read_csv("../../PREPARATION_CSV/Soja_SM_prepared.csv", delimiter=',')
prediction_set = pd.read_csv("../../PREPARATION_CSV/2019Soja_SM.csv", delimiter=',')
cmp = prediction_set.index[0]
size = len(prediction_set)
for i in range(cmp, cmp+size):
    prediction_set.at[i, "Productivite (kg/ha)"]=3300
    prediction_set.at[i, "Coeff_prod"]=3300/3200

In [3]:
FEATURES = ['PRECTOT', 'T2M', 'ALLSKY_SFC_SW_DWN', 'T2MDEW', 'T2M_MIN', 'RH2M', 'YEAR', 'Coeff_prod']
LABEL = 'Productivite (kg/ha)'

In [4]:
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

In [5]:
estimator = tf.estimator.LinearRegressor(feature_columns=feature_cols, model_dir="train")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f542ac80e80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
def get_input_fn(data_set, num_epochs=None, n_batch = 128, shuffle=True):    
         return tf.estimator.inputs.pandas_input_fn(       
         x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),       
         y = pd.Series(data_set[LABEL].values),       
         batch_size=n_batch,          
         num_epochs=num_epochs,       
         shuffle=shuffle)

In [7]:
estimator.train(input_fn=get_input_fn(training_set,                                       
                                           num_epochs=50,                                      
                                           n_batch = 128,                                      
                                           shuffle=False),                                      
                                           steps=1000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into train/model.ckpt.
INFO:tensorflow:loss = 274978800.0, step = 1
INFO:tensorflow:global_step/sec: 426.05
INFO:tensorflow:loss = 29881228.0, step = 101 (0.237 sec)
INFO:tensorf

In [8]:
y = estimator.predict(    
         input_fn=get_input_fn(prediction_set,                          
         num_epochs=50,                          
         n_batch = 128,                          
         shuffle=False))

In [9]:
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from train/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predictions: [array([2177.7483], dtype=float32), array([2184.3694], dtype=float32), array([2240.874], dtype=float32), array([2251.7393], dtype=float32), array([2219.5957], dtype=float32), array([2187.4817], dtype=float32)]


In [10]:
#Marge d'erreur : 3300 - ~2200